#### Machine Learning

In [78]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [79]:
graph_metrics_df = pd.read_csv("../../data/current_graph_metrics.csv")
statuses_df = pd.read_csv("../../data/SpecIDs_and_Status.csv")

In [80]:
df = graph_metrics_df.merge(statuses_df, on="SpecID").sort_values(by="SpecID").set_index('SpecID')
df

,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,LabelPropagation,Leiden,Louvain,Status
SpecID,,,,,,,,
201210-1-00,0.830275,1170.371464,0.014458,0.195683,1,1,2289,Normal
201210-1-01,0.811969,1134.613827,0.014017,0.194311,1,1,2289,Normal
201210-1-02,0.580317,733.700940,0.009064,0.178658,1,127,988,Normal
201210-1-03,0.744123,1017.430766,0.012569,0.189725,1,1,2289,Normal
201210-1-04,0.794119,1107.509457,0.013682,0.193235,1,1,2289,Normal
...,...,...,...,...,...,...,...,...
210526-3-45,1.128517,1689.660278,0.020874,0.215918,1,1,2289,Hyperglycemia
210526-3-46,1.110504,1658.692469,0.020491,0.214711,1,1,2289,Hyperglycemia
210526-3-47,1.145219,1718.172663,0.021226,0.217025,1,1,2289,Hyperglycemia


##### 1. Training a Random Forest and Extra Trees Classifer on the whole spectrum.

In [81]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [82]:
X = df.drop(['Status'], axis=1)
y = df['Status']

et = ExtraTreesClassifier(random_state=1234)

# Performing 10-fold cross-validation for the classifier
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    et.fit(X_train, y_train)
    y_pred = et.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    calculate_metrics(y_test, y_pred)

# Displaying the results
print(f'Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'F1-Score: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')

Overall Accuracy: 0.47540983606557374


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.44      0.47      0.46        91
 Hypoglycemia       0.47      0.50      0.48       107
       Normal       0.52      0.46      0.49       107

     accuracy                           0.48       305
    macro avg       0.48      0.48      0.47       305
 weighted avg       0.48      0.48      0.48       305


Confusion Matrix:
[[43 29 19]
 [28 53 26]
 [27 31 49]]
Overall Accuracy: 0.4721311475409836


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.41      0.42      0.41        91
 Hypoglycemia       0.43      0.42      0.43       107
       Normal       0.56      0.57      0.57       107

     accuracy                           0.47       305
    macro avg       0.47      0.47      0.47       305
 weighted avg       0.47      0.47      0.47       305


Confusion Matrix:
[[38 32 21]
 [36 45 26]


#### Try scaling the features

In [83]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming df is your DataFrame and it has been defined previously

X = df.drop(['Status'], axis=1)
y = df['Status']

et = ExtraTreesClassifier(random_state=1234)

# Performing 10-fold cross-validation for the classifier
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# Initialize lists to store scores
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Scaling the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    et.fit(X_train_scaled, y_train)
    y_pred = et.predict(X_test_scaled)
    
    # Calculate and append the scores
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

# Displaying the results
print(f'Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'F1-Score: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')


Accuracy: 0.4447 +/- 0.0234
Precision: 0.4450 +/- 0.0238
Recall: 0.4447 +/- 0.0234
F1-Score: 0.4443 +/- 0.0235


#### Create a smaller featureset based on feature importance.

In [84]:
# Fitting the Extra Trees classifier
et.fit(X, y)

feature_importances = et.feature_importances_

# Creating a DataFrame to store feature importances with corresponding column names
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sorting the DataFrame by importances in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Selecting the top 100 features
top_3_features = importance_df.head(3)['Feature'].tolist()

# Dropping columns in the DataFrame that are not in the top 100 features
X_top_3= X[top_3_features]


In [85]:
X_top_3

,PageRank,ArticleRank,EigenvectorCentrality
SpecID,,,
201210-1-00,0.830275,0.195683,0.014458
201210-1-01,0.811969,0.194311,0.014017
201210-1-02,0.580317,0.178658,0.009064
201210-1-03,0.744123,0.189725,0.012569
201210-1-04,0.794119,0.193235,0.013682
...,...,...,...
210526-3-45,1.128517,0.215918,0.020874
210526-3-46,1.110504,0.214711,0.020491
210526-3-47,1.145219,0.217025,0.021226


In [86]:
X = X_top_3
et = ExtraTreesClassifier(random_state=1234)

# Performing 10-fold cross-validation for the classifier
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    et.fit(X_train, y_train)
    y_pred = et.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    calculate_metrics(y_test, y_pred)

# Displaying the results
print(f'Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'F1-Score: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')

Overall Accuracy: 0.4852459016393443


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.43      0.43      0.43        91
 Hypoglycemia       0.50      0.53      0.52       107
       Normal       0.51      0.49      0.50       107

     accuracy                           0.49       305
    macro avg       0.48      0.48      0.48       305
 weighted avg       0.49      0.49      0.48       305


Confusion Matrix:
[[39 28 24]
 [24 57 26]
 [27 28 52]]
Overall Accuracy: 0.4557377049180328


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.44      0.41      0.42        91
 Hypoglycemia       0.44      0.42      0.43       107
       Normal       0.48      0.53      0.51       107

     accuracy                           0.46       305
    macro avg       0.45      0.45      0.45       305
 weighted avg       0.45      0.46      0.45       305


Confusion Matrix:
[[37 27 27]
 [28 45 34]
 

>#### Classify based on Node2Vec Embeddings

In [87]:
node2vec_df = pd.read_csv("../../data/node2vec_embeddings.csv")
statuses_df = pd.read_csv("../../data/SpecIDs_and_Status.csv")

In [88]:
df = node2vec_df.merge(statuses_df, on="SpecID").sort_values(by="SpecID").set_index('SpecID')
df

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,0.752320,-0.054122,-0.271341,0.137776,-0.072738,0.039848,-0.045210,0.000818,0.109466,-0.090799,...,0.033261,-0.052994,-0.062999,0.178203,-0.031474,0.049667,0.042749,-0.079675,0.093145,Normal
201210-1-01,0.760893,-0.042880,-0.269445,0.143065,-0.076744,0.034551,-0.052938,0.005371,0.107738,-0.090808,...,0.032457,-0.049853,-0.063992,0.183698,-0.021917,0.024821,0.050071,-0.075491,0.083714,Normal
201210-1-02,0.702787,-0.033687,-0.246922,0.130849,-0.063006,0.030563,-0.031748,-0.001555,0.100316,-0.082333,...,0.033401,-0.058833,-0.052605,0.165816,-0.005158,0.043624,0.052950,-0.065073,0.083130,Normal
201210-1-03,0.747901,-0.051093,-0.262347,0.136324,-0.064270,0.027572,-0.045275,0.010077,0.110294,-0.083121,...,0.037874,-0.058256,-0.056513,0.187060,-0.032658,0.040483,0.034975,-0.077696,0.093038,Normal
201210-1-04,0.727759,-0.040931,-0.248123,0.125667,-0.066284,0.024941,-0.037412,-0.014402,0.095174,-0.094227,...,0.031387,-0.052911,-0.049742,0.177678,-0.014392,0.038045,0.044815,-0.075423,0.080636,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-45,0.673561,-0.038367,-0.231138,0.114172,-0.059030,0.030855,-0.033409,-0.043275,0.084705,-0.082340,...,0.042226,-0.053605,-0.056190,0.156956,0.025843,0.027423,0.050027,-0.073579,0.066306,Hyperglycemia
210526-3-46,0.702694,-0.047516,-0.239269,0.135926,-0.047491,0.020640,-0.027082,-0.011145,0.095510,-0.085104,...,0.038394,-0.054875,-0.052379,0.167737,-0.005255,0.023794,0.040700,-0.074320,0.077314,Hyperglycemia
210526-3-47,0.731247,-0.049377,-0.252198,0.141808,-0.064303,0.028521,-0.032889,-0.017582,0.110007,-0.093290,...,0.054310,-0.063347,-0.057416,0.158384,0.006886,0.042873,0.046079,-0.064566,0.082226,Hyperglycemia


In [89]:
X = df.drop(['Status'], axis=1)
y = df['Status']

et = ExtraTreesClassifier(random_state=1234)

# Performing 10-fold cross-validation for the classifier
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    et.fit(X_train, y_train)
    y_pred = et.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    calculate_metrics(y_test, y_pred)

# Displaying the results
print(f'Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'F1-Score: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')

Overall Accuracy: 0.40327868852459015


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.36      0.25      0.30        91
 Hypoglycemia       0.47      0.47      0.47       107
       Normal       0.37      0.47      0.41       107

     accuracy                           0.40       305
    macro avg       0.40      0.40      0.39       305
 weighted avg       0.40      0.40      0.40       305


Confusion Matrix:
[[23 21 47]
 [20 50 37]
 [21 36 50]]
Overall Accuracy: 0.419672131147541


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.44      0.34      0.39        91
 Hypoglycemia       0.40      0.42      0.41       107
       Normal       0.42      0.49      0.45       107

     accuracy                           0.42       305
    macro avg       0.42      0.42      0.42       305
 weighted avg       0.42      0.42      0.42       305


Confusion Matrix:
[[31 29 31]
 [22 45 40]
 

>#### Classify based on FastRP Embeddings

In [98]:
fastRP_df = pd.read_csv("../../data/fastRP_embeddings.csv")
statuses_df = pd.read_csv("../../data/SpecIDs_and_Status.csv")

In [91]:
df = fastRP_df.merge(statuses_df, on="SpecID").sort_values(by="SpecID").set_index('SpecID')
df

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,-0.080063,0.134346,0.311409,0.068268,0.220651,0.020809,-0.157989,-0.023693,0.139395,0.119681,...,0.062691,-0.112076,0.184696,0.327179,-0.075120,0.091235,-0.209512,0.040763,-0.094208,Normal
201210-1-01,-0.080101,0.134585,0.310871,0.067332,0.220452,0.020230,-0.158860,-0.023553,0.139669,0.118961,...,0.063693,-0.111742,0.186010,0.326752,-0.076596,0.091260,-0.211565,0.040145,-0.094633,Normal
201210-1-02,-0.079054,0.138156,0.306293,0.055854,0.212508,0.017010,-0.174445,-0.019908,0.143086,0.116697,...,0.065156,-0.112998,0.196053,0.328230,-0.083976,0.093691,-0.220491,0.040576,-0.086324,Normal
201210-1-03,-0.079920,0.136971,0.307921,0.059693,0.213916,0.018670,-0.169574,-0.020787,0.141887,0.117424,...,0.063974,-0.112868,0.192851,0.328275,-0.082046,0.093300,-0.217178,0.040930,-0.088214,Normal
201210-1-04,-0.079154,0.133878,0.312250,0.070663,0.223848,0.020361,-0.154181,-0.024949,0.138894,0.120166,...,0.062828,-0.111242,0.182163,0.326090,-0.072420,0.090532,-0.207949,0.039952,-0.096966,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-45,-0.077160,0.133637,0.313199,0.071438,0.226589,0.019494,-0.154102,-0.025647,0.139182,0.121855,...,0.062112,-0.111478,0.180060,0.326621,-0.067590,0.089882,-0.204368,0.040522,-0.096711,Hyperglycemia
210526-3-46,-0.077198,0.133361,0.313462,0.072150,0.227169,0.019669,-0.153159,-0.025882,0.138980,0.122001,...,0.062088,-0.111391,0.179395,0.326495,-0.067051,0.089678,-0.203765,0.040492,-0.097252,Hyperglycemia
210526-3-47,-0.076899,0.134395,0.312508,0.069708,0.225286,0.019181,-0.156088,-0.025093,0.139682,0.121436,...,0.062091,-0.111497,0.181595,0.326650,-0.068859,0.090412,-0.205910,0.040582,-0.095570,Hyperglycemia


In [92]:
X = df.drop(['Status'], axis=1)
y = df['Status']

et = ExtraTreesClassifier(random_state=1234)

# Performing 10-fold cross-validation for the classifier
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    et.fit(X_train, y_train)
    y_pred = et.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    calculate_metrics(y_test, y_pred)

# Displaying the results
print(f'Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'F1-Score: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')

Overall Accuracy: 0.7934426229508197


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.72      0.74      0.73        91
 Hypoglycemia       0.81      0.78      0.79       107
       Normal       0.84      0.86      0.85       107

     accuracy                           0.79       305
    macro avg       0.79      0.79      0.79       305
 weighted avg       0.79      0.79      0.79       305


Confusion Matrix:
[[67 15  9]
 [15 83  9]
 [11  4 92]]
Overall Accuracy: 0.7672131147540984


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.73      0.79      0.76        91
 Hypoglycemia       0.76      0.69      0.73       107
       Normal       0.81      0.82      0.81       107

     accuracy                           0.77       305
    macro avg       0.77      0.77      0.77       305
 weighted avg       0.77      0.77      0.77       305


Confusion Matrix:
[[72 12  7]
 [19 74 14]
 

Try Fast RP with the full Scaled Spectra.

In [93]:
scaled_spectra = pd.read_csv("../../data/scaled_raw_spectra.csv")

In [94]:
scaled_spectra.head()

,SpecID,200.0,200.68336,201.36674,202.05011,202.73349,203.41685,204.10023,204.7836,205.46696,...,1994.5331,1995.2164,1995.8998,1996.5831,1997.2665,1997.9498,1998.6333,1999.3167,2000.0,Status
0,201210-1-00,1.000000,0.995483,0.995081,0.988640,0.985799,0.979026,0.976731,0.992970,0.967134,...,0.406183,0.401843,0.403086,0.407818,0.400140,0.397486,0.401969,0.403344,0.401089,Normal
1,201210-1-01,0.997860,1.000000,0.994795,0.987068,0.985319,0.985517,0.984678,0.984173,0.981650,...,0.417414,0.425509,0.426087,0.425988,0.425384,0.430038,0.431047,0.427505,0.424721,Normal
2,201210-1-02,1.000000,0.970271,0.983330,0.970521,0.967885,0.967837,0.959964,0.947223,0.943806,...,0.547909,0.552820,0.565779,0.558489,0.551799,0.557342,0.563581,0.557661,0.557509,Normal
3,201210-1-03,0.785427,0.794385,0.808680,0.824538,0.839005,0.850751,0.866092,0.890804,0.894421,...,0.162540,0.163745,0.165235,0.165661,0.168732,0.164766,0.164548,0.164754,0.162991,Normal
4,201210-1-04,0.998059,0.996477,1.000000,0.998082,0.993222,0.989748,0.989609,0.991243,0.991758,...,0.496500,0.495582,0.505587,0.502293,0.497229,0.503632,0.502699,0.505335,0.502975,Normal


In [99]:
df = fastRP_df.merge(scaled_spectra, on="SpecID").sort_values(by="SpecID").set_index('SpecID')
df.head()

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,1994.5331,1995.2164,1995.8998,1996.5831,1997.2665,1997.9498,1998.6333,1999.3167,2000.0,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,-0.080063,0.134346,0.311409,0.068268,0.220651,0.020809,-0.157989,-0.023693,0.139395,0.119681,...,0.406183,0.401843,0.403086,0.407818,0.400140,0.397486,0.401969,0.403344,0.401089,Normal
201210-1-01,-0.080101,0.134585,0.310871,0.067332,0.220452,0.020230,-0.158860,-0.023553,0.139669,0.118961,...,0.417414,0.425509,0.426087,0.425988,0.425384,0.430038,0.431047,0.427505,0.424721,Normal
201210-1-02,-0.079054,0.138156,0.306293,0.055854,0.212508,0.017010,-0.174445,-0.019908,0.143086,0.116697,...,0.547909,0.552820,0.565779,0.558489,0.551799,0.557342,0.563581,0.557661,0.557509,Normal
201210-1-03,-0.079920,0.136971,0.307921,0.059693,0.213916,0.018670,-0.169574,-0.020787,0.141887,0.117424,...,0.162540,0.163745,0.165235,0.165661,0.168732,0.164766,0.164548,0.164754,0.162991,Normal
201210-1-04,-0.079154,0.133878,0.312250,0.070663,0.223848,0.020361,-0.154181,-0.024949,0.138894,0.120166,...,0.496500,0.495582,0.505587,0.502293,0.497229,0.503632,0.502699,0.505335,0.502975,Normal


In [100]:
X = df.drop(['Status'], axis=1)
y = df['Status']

et = ExtraTreesClassifier(random_state=1234)

# Performing 10-fold cross-validation for the classifier
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    et.fit(X_train, y_train)
    y_pred = et.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    calculate_metrics(y_test, y_pred)

# Displaying the results
print(f'Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'F1-Score: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')

Overall Accuracy: 0.9180327868852459


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.91      0.91      0.91        91
 Hypoglycemia       0.92      0.93      0.93       107
       Normal       0.92      0.91      0.92       107

     accuracy                           0.92       305
    macro avg       0.92      0.92      0.92       305
 weighted avg       0.92      0.92      0.92       305


Confusion Matrix:
[[ 83   3   5]
 [  4 100   3]
 [  4   6  97]]
Overall Accuracy: 0.9836065573770492


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.99      0.99      0.99        91
 Hypoglycemia       0.99      0.97      0.98       107
       Normal       0.97      0.99      0.98       107

     accuracy                           0.98       305
    macro avg       0.98      0.98      0.98       305
 weighted avg       0.98      0.98      0.98       305


Confusion Matrix:
[[ 90   1   0]
 